In [ ]:
import pandas as pd
import os

# Correct relative path
base_path = "../Data/csv/"

# Confirm the files exist
for file in ["leases.csv", "locations.csv", "properties.csv", "tenants.csv", "units.csv"]:
    full_path = os.path.join(base_path, file)
    print(file, "→", os.path.exists(full_path))

# Load the CSVs
leases = pd.read_csv(os.path.join(base_path, "leases.csv"))
locations = pd.read_csv(os.path.join(base_path, "locations.csv"))
properties = pd.read_csv(os.path.join(base_path, "properties.csv"))
tenants = pd.read_csv(os.path.join(base_path, "tenants.csv"))
units = pd.read_csv(os.path.join(base_path, "units.csv"))

# Quick preview of the dataset
for name, df in zip(["leases", "locations", "properties", "tenants", "units"],
                    [leases, locations, properties, tenants, units]):
    print(f"\n{name.upper()} --> {df.shape}")
    print(df.head(2))


leases.csv → True
locations.csv → True
properties.csv → True
tenants.csv → True
units.csv → True

LEASES --> (12, 7)
   id  unit_id  tenant_id  rent_per_month  arrears  start_date    end_date
0   1        1          1           45000        0  01/01/2024         NaN
1   2        2          2           55000     5000  01/11/2023  31/10/2024

LOCATIONS --> (3, 1)
         id;name
0  1;Nairobi CBD
1    2;Westlands

PROPERTIES --> (5, 1)
  id;name;location_id
0     1;NSSF Towers;1
1    2;Delta Corner;2

TENANTS --> (15, 1)
                             id;name;email
0  1;Amina Mwangi;amina.mwangi@example.com
1  2;Brian Otieno;brian.otieno@example.com

UNITS --> (20, 1)
  id;property_id;name;size
0           1;1;A-101;75.0
1           2;1;A-102;80.5


In [6]:
# Re-load with correct delimiter detection
leases = pd.read_csv(os.path.join(base_path, "leases.csv"))  # normal comma-separated
locations = pd.read_csv(os.path.join(base_path, "locations.csv"), sep=';')
properties = pd.read_csv(os.path.join(base_path, "properties.csv"), sep=';')
tenants = pd.read_csv(os.path.join(base_path, "tenants.csv"), sep=';')
units = pd.read_csv(os.path.join(base_path, "units.csv"), sep=';')

for name, df in zip(["leases", "locations", "properties", "tenants", "units"],
                    [leases, locations, properties, tenants, units]):
    print(f"\n{name.upper()} --> {df.shape}")
    print(df.head(2))



LEASES --> (12, 7)
   id  unit_id  tenant_id  rent_per_month  arrears  start_date    end_date
0   1        1          1           45000        0  01/01/2024         NaN
1   2        2          2           55000     5000  01/11/2023  31/10/2024

LOCATIONS --> (3, 2)
   id         name
0   1  Nairobi CBD
1   2    Westlands

PROPERTIES --> (5, 3)
   id          name  location_id
0   1   NSSF Towers            1
1   2  Delta Corner            2

TENANTS --> (15, 3)
   id          name                     email
0   1  Amina Mwangi  amina.mwangi@example.com
1   2  Brian Otieno  brian.otieno@example.com

UNITS --> (20, 4)
   id  property_id   name  size
0   1            1  A-101  75.0
1   2            1  A-102  80.5


In [7]:
for name, df in zip(
    ["leases", "locations", "properties", "tenants", "units"],
    [leases, locations, properties, tenants, units]
):
    print(f"\n{name.upper()} columns:")
    print(df.columns.tolist())



LEASES columns:
['id', 'unit_id', 'tenant_id', 'rent_per_month', 'arrears', 'start_date', 'end_date']

LOCATIONS columns:
['id', 'name']

PROPERTIES columns:
['id', 'name', 'location_id']

TENANTS columns:
['id', 'name', 'email']

UNITS columns:
['id', 'property_id', 'name', 'size']


In [24]:
# Merge properties with locations
prop_loc = properties.merge(
    locations,
    left_on="location_id",
    right_on="id",
    suffixes=("_property", "_location"),
    how="outer"
)

# Merge units with properties + locations
unit_prop_loc = units.merge(
    prop_loc,
    left_on="property_id",
    right_on="id_property",
    how="outer"
)

# Merge leases with tenants
lease_tenant = leases.merge(
    tenants,
    left_on="tenant_id",
    right_on="id",
    suffixes=("_lease", "_tenant"),
    how="outer"
)

# Merge everything together
merged_df = lease_tenant.merge(
    unit_prop_loc,
    left_on="unit_id",
    right_on="id"
)

# Drop redundant ID columns
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

print("Final merged shape:", merged_df.shape)
merged_df.head(5)


Final merged shape: (12, 19)


,id_lease,unit_id,tenant_id,rent_per_month,arrears,start_date,end_date,id_tenant,name_x,email,id,property_id,name_y,size,id_property,name_property,location_id,id_location,name_location
0,1.0,1.0,1.0,45000.0,0.0,01/01/2024,NaN,1,Amina Mwangi,amina.mwangi@example.com,1,1,A-101,75.0,1,NSSF Towers,1,1,Nairobi CBD
1,2.0,2.0,2.0,55000.0,5000.0,01/11/2023,31/10/2024,2,Brian Otieno,brian.otieno@example.com,2,1,A-102,80.5,1,NSSF Towers,1,1,Nairobi CBD
2,10.0,10.0,2.0,-45000.0,1000.0,01/02/2024,31/12/2024,2,Brian Otieno,brian.otieno@example.com,10,2,DC-3A,73.5,2,Delta Corner,2,2,Westlands
3,3.0,3.0,3.0,65000.0,-2000.0,01/02/2024,NaN,3,Carol Wanjiru,carol.wanjiru@example.com,3,1,B-201,65.0,1,NSSF Towers,1,1,Nairobi CBD
4,5.0,5.0,3.0,70000.0,0.0,01/02/2025,31/12/2025,3,Carol Wanjiru,carol.wanjiru@example.com,5,1,C-301,120.0,1,NSSF Towers,1,1,Nairobi CBD


In [26]:
clean_df = merged_df.copy()

# Rename columns for clarity
clean_df = clean_df.rename(columns={
    "id_lease": "lease_id",
    "name_x": "tenant_name",
    "name_y": "unit_name",
    "name_property": "property_name",
    "name_location": "location_name"
})

# Drop redundant ID columns
cols_to_drop = [
    "tenant_id", "id_tenant", "id", "property_id",
    "id_property", "id_location", "location_id"
]
clean_df = clean_df.drop(columns=cols_to_drop, errors="ignore")

clean_df = clean_df[
    [
        "lease_id", "tenant_name", "email", "unit_name", "size",
        "property_name", "location_name",
        "rent_per_month", "arrears", "start_date", "end_date"
    ]
]

print("Cleaned shape:", clean_df.shape)
clean_df.head(5)


Cleaned shape: (12, 11)


,lease_id,tenant_name,email,unit_name,size,property_name,location_name,rent_per_month,arrears,start_date,end_date
0,1.0,Amina Mwangi,amina.mwangi@example.com,A-101,75.0,NSSF Towers,Nairobi CBD,45000.0,0.0,01/01/2024,NaN
1,2.0,Brian Otieno,brian.otieno@example.com,A-102,80.5,NSSF Towers,Nairobi CBD,55000.0,5000.0,01/11/2023,31/10/2024
2,10.0,Brian Otieno,brian.otieno@example.com,DC-3A,73.5,Delta Corner,Westlands,-45000.0,1000.0,01/02/2024,31/12/2024
3,3.0,Carol Wanjiru,carol.wanjiru@example.com,B-201,65.0,NSSF Towers,Nairobi CBD,65000.0,-2000.0,01/02/2024,NaN
4,5.0,Carol Wanjiru,carol.wanjiru@example.com,C-301,120.0,NSSF Towers,Nairobi CBD,70000.0,0.0,01/02/2025,31/12/2025


In [28]:
# Convert any negative rent_per_month to positive
clean_df["rent_per_month"] = clean_df["rent_per_month"].abs()
clean_df.head(15)

,lease_id,tenant_name,email,unit_name,size,property_name,location_name,rent_per_month,arrears,start_date,end_date
0,1.0,Amina Mwangi,amina.mwangi@example.com,A-101,75.0,NSSF Towers,Nairobi CBD,45000.0,0.0,01/01/2024,NaN
1,2.0,Brian Otieno,brian.otieno@example.com,A-102,80.5,NSSF Towers,Nairobi CBD,55000.0,5000.0,01/11/2023,31/10/2024
2,10.0,Brian Otieno,brian.otieno@example.com,DC-3A,73.5,Delta Corner,Westlands,45000.0,1000.0,01/02/2024,31/12/2024
3,3.0,Carol Wanjiru,carol.wanjiru@example.com,B-201,65.0,NSSF Towers,Nairobi CBD,65000.0,-2000.0,01/02/2024,NaN
4,5.0,Carol Wanjiru,carol.wanjiru@example.com,C-301,120.0,NSSF Towers,Nairobi CBD,70000.0,0.0,01/02/2025,31/12/2025
5,4.0,David Kiptoo,david.kiptoo@example.com,B-202,92.0,NSSF Towers,Nairobi CBD,30000.0,1000.0,01/03/2024,NaN
6,6.0,Eunice Njeri,eunice.njeri@example.com,DC-1A,55.0,Delta Corner,Westlands,40000.0,8000.0,15/06/2024,14/06/2025
7,7.0,Farah Hassan,farah.hassan@example.com,DC-1B,60.0,Delta Corner,Westlands,52000.0,0.0,01/08/2024,31/07/2024
8,8.0,George Ouma,george.ouma@example.com,DC-2A,85.0,Delta Corner,Westlands,38000.0,12000.0,01/09/2023,NaN
9,9.0,Hannah Achieng,hannah.achieng@example.com,DC-2B,100.0,Delta Corner,Westlands,60000.0,0.0,01/01/2025,NaN
